In [12]:
from utils import count_nonzero_parameters, load_untrained_model

In [15]:
dict_ = load_untrained_model("DenseNet121")
model = dict_["model"]

6914538

In [1]:
import pickle

with open("train_results_increasing_grouped_densenet121.pkl", "rb") as f:
    res = pickle.load(f)

In [13]:
from factorisation.densenet import get_increasing_grouped_densenet121


model = get_increasing_grouped_densenet121()
count_nonzero_parameters(model)

897002

In [5]:
with open("train_results_increasing_transition_grouped_densenet121.pkl", "rb") as f:
    res = pickle.load(f)

res["acc"]

78.66

In [6]:
with open("train_results_transition_grouped_densenet121.pkl", "rb") as f:
    res = pickle.load(f)

res["acc"]

90.37